三、模型输出和损失函数的选择

接下来需要解决的问题就是，模型该输出什么样的结果呢？

1. 首先一个最自然的想法就是，既然我们的预测目标是连续值，直接让模型输出连续的预测结果，然后用MSELoss计算损失。

    然而，经过实践发现，主要有两点问题：

    (1). 由于金融数据含有大量的噪声，即使是相同的市场状态，未来的变化也有很大变数，即市场的混沌性。

        此时如果我们用MSE计算损失，模型为了避免预测错误带来的平方高额损失，会倾向于预测一个比较稳健的平均值来减小损失，导致模型预测的结果围绕在0周围，往往都是非常小的数字；

    (2). 虽然预测值是离散的，但是我们的交易操作却是离散的。

        如果直接根据预测结果的符号进行交易，再叠加噪声的影响，会很难判断模型的真正预测。
    
        比如模型给出的+0.2的预测，这个数字非常小，是否要根据这样的预测进行交易呢？

    此外，尝试改为使用MAELoss(L1Loss)，虽然预测值的绝对值很小的问题有所缓解，但依然存在。

2. 那么，能否改为分类问题 + BCELoss，将真实值离散为上涨、下跌两个状态，让模型只预测方向呢？

    经过实验，发现前述问题得到了缓解，但是仍然有两个新问题：

    (1). 真实值离散化之后，丢失了信息，+2和+200是完全不同的结果，模型理应区别对之；

    (2). 在训练过程中，模型需要一些时间来学习上涨形态和下跌形态的特征，但是如果在此之前，logits的差距已经很大，会导致经softmax之后的prob一边倒。
    
        这样的情况下，即使模型学会了某个特征，但由于此时prob已经很大或很小，梯度几乎消失，无法反向传播更新参数来传递这个学习到的特征，导致训练失败。

3. 结合上述的问题，我最终提出了波动率感知的概率标签损失函数。主要从以下几个方面解决上述问题：

    (1). 从二分类改为三分类：回想人类的交易决策，一个科学的交易策略应当是三分类的，在行情不明确的时候，要有退出市场的选项来规避预期外的风险。行情不明确，无法预测的时候入场，只会增大风险，降低sharpe ratio。

        因此，我们的模型不应当在任何时候都给出预测，而是应当学会放弃一部分自己无法预测的情况，

    (2). 参考分类问题的 “软标签” ，即使在训练初期模型预测一边倒的情况下，避免模型过拟合风险，同时也能保持梯度流动，增强训练过程的稳定性

    (3). 但相比普通的固定数值的软标签，我们可以通过引入波动率变量的形式，对每个样本生成独立的、更加科学的软标签，这一点是本损失函数的关键要点：

在波动率感知的概率标签损失函数中，我们假定：

每个窗口存在一个隐含收益；

由于噪声和混沌的影响，真实收益会在隐含收益的基础上附加一个白噪声；

由于我们只能观测到真实收益，所以隐含收益是不可知的，但由于真实收益已知，隐含收益落在不同的区间内的概率是不一致的；

    （例如，假设某日观测到的真实收益是-50，那么隐含收益落在(-inf, 0)区间的概率就显然大于落在(0, +inf)区间的概率）

因此隐含收益服从n=1时的t分布

然而，我们并没有关于收益标准差的信息：

对此，可以使用与预测目标同窗口的成交量数据，作为隐含波动率的参考，因为价格的波动需要成交量作为驱动，对于大盘尤其是如此。

最终我们可以设计如下的损失函数：

首先确定一个放弃预测的阈值，真实值绝对值小于阈值时，认为是波动较小、趋势不明显，应当放弃交易。可以参考预测目标的一个标准差范围。

对于任意一个样本，模型给出3个logits，分别代表预测下跌，放弃预测，预测上涨；size = (batch_size, 3)

对于真实值作为均值，以成交量（scale之后）作为标准差，计算正态分布下(-inf, -threshold), (-threshold, +threshold), (+threshold, +inf) 三个区间的概率分布；size = (batch_size, 3)

(这里不采用t分布的原因是为了简化梯度计算，因为标准差也是成交量估计出来的，本身误差也很大，在这样的误差之下正态分布和t分布的差距可以忽略了)

对于三个logits 应用log_softmax 之后，和三个概率分布计算KL散度，评估两个分布的差距来作为该样本的预测损失。

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Normal

class VAPLLoss(nn.Module):
    """
    波动率感知的概率标签损失函数（Volatility-aware Probabilistic Label Loss Function）
    应用于金融资产收益率标签预测的损失函数。
    根据涨跌幅和标准差，动态生成概率软标签，并用KL散度计算损失。
    避免离散化为标签时丢失信息，同时利用成交量等波动率参数强化预测效果。
    支持二分类和三分类。
    """
    def __init__(self, num_classes: int, threshold: float = 0.05, is_logits: bool = True):
        super(VAPLLoss, self).__init__()
        if num_classes not in [2, 3]:
            raise ValueError("num_classes must be 2 or 3.")
        self.num_classes = num_classes
        self.threshold = threshold
        self.is_logits = is_logits
        self.kl_loss = nn.KLDivLoss(reduction='batchmean')

    def forward(self, pred: torch.Tensor, real: torch.Tensor, std: torch.Tensor):

        # 维度检查
        if std.dim() == 0 or (std.dim() == 1 and std.size(0) == 1):
            std = std.expand_as(real)
        assert pred.shape[0] == real.shape[0], "Batch sizes of pred and real must match."
        assert pred.shape[1] == self.num_classes, f"Pred tensor must have {self.num_classes} classes."

        # 用正态分布近似t分布来简化梯度计算
        dist = Normal(loc=real, scale=std)

        if self.num_classes == 2:
            prob_positive = 1 - dist.cdf(torch.tensor(0.0, device=real.device))
            prob_negative = dist.cdf(torch.tensor(0.0, device=real.device))
            soft_labels = torch.cat([prob_positive, prob_negative], dim=-1)
            
        elif self.num_classes == 3:
            prob_negative = dist.cdf(torch.tensor(-self.threshold, device=real.device))
            prob_neutral = dist.cdf(torch.tensor(self.threshold, device=real.device)) - prob_negative
            prob_positive = 1 - dist.cdf(torch.tensor(self.threshold, device=real.device))
            soft_labels = torch.cat([prob_negative, prob_neutral, prob_positive], dim=-1)

        soft_labels = soft_labels / soft_labels.sum(dim=-1, keepdim=True)

        if self.is_logits:
            log_pred = F.log_softmax(pred, dim=1)
        else:
            # 避免log(0)，增加一个极小的常数epsilon
            epsilon = 1e-9
            log_pred = torch.log(pred + epsilon)
        

        loss = self.kl_loss(log_pred, soft_labels)
        
        return loss